<a href="https://colab.research.google.com/github/WiemHAD/Brief_NLP/blob/main/rephrase_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Downloading librairies**

In [3]:
! pip install credentials
! pip install config

In [4]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-jqrm1aij
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-jqrm1aij
     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 1.2 MB 40.0 MB/s 
     |████████████████████████████████| 50 kB 5.1 MB/s 
     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 636 kB 47.6 MB/s 
     |████████████████████████████████| 895 kB 24.9 MB/s 
     |████████████████████████████████| 3.3 MB 40.2 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8695 sha256=5773a12b1b507b52644adfbc96e1d0afd3cc864d89f81ac99d82ea569277de15
  Stored in directory: /tmp/pip-ephem-wheel-cache-faurwxjs/wheels/d8/27/d8/b757755a5b0b804c59714fdd8a4e2733c20802f638f17f7d96
  Created wheel for python-Levenshtein: filename=python_Levenshtein-

**Import librairies**

In [5]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import tweepy as tw
import config
from credentials import *
import re  # For preprocessing
import nltk
nltk.download('punkt')
from nltk import tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
# For reproducibility
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(100)

In [ ]:
#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=True)

**class Rephrase:**
 - parametres: user_Id and the index of tweet to rephrase
 - output: the rephrased sentences composing the tweet
 - methodes: 

1.    *scraping*: make scraping of 200 last tweets. can provide tweet_id, user_id and the time when the tweet was published.
* input: None
* output: tweet with the chosen index

2.   *preprocessing*: make basic preprocessing steps: remove links, hashtags and tags using regex and split the tweet in sentences. 
* input: the chosen tweet (str)
* output: list of sentences composing the chosen tweet

3. *rephrasing*: rephrase sentences using a parrot model
* input: list of sentences composing the chosen tweet
* output: list of rephrased sentences sorted according a score which evaluate the quality of rephrasing 

In [ ]:
class Rephrase:
  def __init__(self,userID,tweet_index=0):
    self.userID = userID
    self.tweet_index = tweet_index
    self.chosen_tweet = None
    self.consumer_key= 'HxklRvkv2hhS2AUkXhEQBxpvZ'
    self.consumer_secret= 'UEgBbSDtQzAty1EJjmy4Vjo3oAKwYfhQDkrkhRlLpk0BhCYmsv'
    self.access_token = '1409627133253718024-StgjAUt98nEYufIr3qfm0KoCVfEkR0'
    self.access_token_secret = 'AnmmTQZWxndzXu1lsSCNyrCM1BwCVA12aF0Eet48sx7Ft'
          

  def scraping(self):
    auth = tw.OAuthHandler(self.consumer_key, self.consumer_secret)
    auth.set_access_token(self.access_token, self.access_token_secret)
    api = tw.API(auth)
    tweets = api.user_timeline(screen_name=self.userID, 
                           # 200 is the maximum allowed count
                           count= 200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    
    self.chosen_tweet = tweets[self.tweet_index].full_text
    print("original tweet : ", self.chosen_tweet)


  def preprocessing(self):
    regex1 = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    regex2 = '\@[A-Za-z0-9.-]+'
    regex3 = '\B(\#[a-zA-Z]+\b)(?!;)'
    sentences = re.sub(regex1, '', self.chosen_tweet, flags=re.MULTILINE)
    sentences = re.sub(regex2, '', sentences, flags=re.MULTILINE)
    sentences = re.sub(regex3, '', sentences, flags=re.MULTILINE)

    self.phrases = tokenize.sent_tokenize(sentences) #transform tweet on a list of sentences


  def rephrasing(self):
    with open('recap.txt', 'w') as f:
      for txt in self.phrases:
        print("-"*100)
        print("Input_phrase: ", txt)
        print("-"*100)
        f.write(txt)
        f.write('\n')
        para_phrases = parrot.augment(input_phrase=txt,
                                      use_gpu=False,
                                      diversity_ranker="levenshtein",
                                      do_diverse=False, 
                                      max_return_phrases = 10,
                                      max_length=32,
                                      adequacy_threshold = 0.99,
                                      fluency_threshold = 0.90)
        
        for para_phrase in para_phrases:
          print(para_phrase)
          f.write(str(para_phrase))
          f.write('\n')

  def run(self):
    self.scraping()
    self.preprocessing()
    self.rephrasing()  

In [ ]:
test = Rephrase('@BarackObama',tweet_index=0)
test.run()